以一个实例实现Qwen2模型的微调流程。理解大模型整个微调的过程，学习数据的处理过程、微调的数据格式，以及在微调过程中使用到的相关技术、工具的基本使用。
<br>
如果微调qwen2-0.5b-instruct，则对显存需求较小，个人PC即可完成。
<br>
本文档以医疗对话数据集和自我认知数据集为例，进行展示。

#### 导入相关库

In [1]:
import os
from transformers import (AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, TrainingArguments, Trainer, pipeline, logging,)
from peft import LoraConfig
import torch
#os.environ['CUDA_VISIBLE_DEVICES']='0'
device_map = {"": 0} 
device = 'cuda'


/opt/envs/miniconda3/envs/pytorch2.1/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### 数据处理
医疗数据集下载地址：https://github.com/Toyhom/Chinese-medical-dialogue-data
<br>
说明：
<br>
该数据包含了<Andriatria_男科> 94596个问答对   <IM_内科> 220606个问答对     <OAGD_妇产科>
183751个问答对     <Oncology_肿瘤科> 75553个问答对    <Pediatric_⼉科> 101602个问答对     <Surgical_
外科> 115991个问答对 总计 792099条。
<br>
<br>
自我认识数据集：https://github.com/SmartFlowAI/Llama3-Tutorial/blob/main/data/self_cognition.json
<br>
在实际使用的时候需要将数据中的name和author替换为我们自己机器人的名字，并且可以根据我们实际使用场景修改一些必要的回答。

In [2]:
def load_dataset(filename):
    data_list = []
    with open(filename, "r", encoding="gb18030") as f:
        i = 0
        for line in f:
            i += 1
            if i < 5:  # 打印几条数据示例
                print(line)
            try:
                dept, title, ques, ans = line.strip("\n").split(',', 4)
                data_list.append(
                    {
                    'department': dept,
                    'input': ques,
                    'output': ans
                    }
                )
            except:
                pass
            #if i<5:
            #    print(data_list[i-1])
    return data_list

In [3]:
# 仅加载儿科数据集为例，对于机器充足的同学可以全部加载
data_list = load_dataset("../../data/llm/Chinese-medical-dialogue-data-master/Data_数据/Pediatric_儿科/儿科5-14000.csv")
print(len(data_list))   # 查看数据长度

department,title,ask,answer

营养保健科,小儿肥胖超重该如何治疗,女宝宝，刚7岁，这一年，察觉到，我家孩子身上肉很多，而且，食量非常的大，平时都不喜欢吃去玩，请问：小儿肥胖超重该如何治疗。,孩子出现肥胖症的情况。家长要通过孩子运功和健康的饮食来缓解他的症状，可以先让他做一些有氧运动，比如慢跑，爬坡，游泳等，并且饮食上孩子多吃黄瓜，胡萝卜，菠菜等，禁止孩子吃一些油炸食品和干果类食物，这些都是干热量高脂肪的食物，而且不要让孩子总是吃完就躺在床上不动，家长在治疗小儿肥胖期间如果孩子情况严重就要及时去医院在医生的指导下给孩子治疗。

营养保健科,小儿肥胖超重该怎样医治,男孩子，刚4岁，最近，发现，我家孩子体重要比别的孩子重很多，而且，最近越来越能吃了，还特别的懒，请问：小儿肥胖超重该怎样医治。,孩子一旦患上肥胖症家长要先通过运动和饮食来改变孩子的情况，要让孩子做一些他这个年龄段能做的运动，如游泳，慢跑等，要给孩子多吃一些像苹果，猕猴桃，胡萝卜等食物，禁止孩子吃高热量，高脂肪的食物，像蛋糕，干果，曲奇饼干等，严格的控制孩子的饮食，不要让他暴饮暴食，多运动对改变孩子肥胖都是有好处的，在治疗小儿肥胖期间如果情况严重，建议家长先带孩子去医院检查一下孩子肥胖症的原因在针对性的治疗。

营养保健科,小儿肥胖能吃该如何治疗,男宝，已经5岁，今年，察觉到，孩子身上越来越肉乎了，同时，吃的饭也比一般孩子多，平时都不喜欢吃去玩，请问：小儿肥胖能吃该如何治疗。,当孩子患上肥胖症的时候家长可以增加孩子的运动量和控制他的饮食来改变症状，像游泳，爬坡这类游泳运动对肥胖的症状都很好的效果，像冬瓜，西红柿这样高纤维的蔬菜要多吃一些，孩子不可以吃像蛋糕，夏威夷果这些高热量的食物，而且不要让孩子总是吃完就躺在床上不动，家长在治疗小儿肥胖期间如果孩子情况严重就要及时去医院在医生的指导下给孩子治疗。

84344


我们将数据格式转化为这样的数据格式：

In [4]:
def prepare_message(data_list):
    '''
    格式样例：
    [
        {
            "type": "chatml",
            "conversations": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": "你好"
                },
                {
                    "role": "assistant",
                    "content": "我是一个语言模型，我叫通义千问。"
                }
            ],
            "source": "self_cong"
        }
    ]
    '''
    new_list = []
    for i, data in enumerate(data_list):
        new_list.append(
            {
                "type": "chatml",
                "messages": [
                    {
                        "role": "system",
                        "content": "You are a helpful assistant."
                    },
                    {
                        "role": "user",
                        "content": data["input"]
                    },
                    {
                        "role": "assistant",
                        "content": data["output"]
                    }
                ],
                "source": "medical"            
            }
        )
    return new_list

接下来，我们准备自我认知数据self_cognition.json的载入和转化格式。
首先定义替换和的函数，可以根据需求更改名字。

In [5]:
# 修改机器人的名字，可以自由替换
def replace_name(s):   
    s = s.replace('<NAME>', '智能医学客服机器人')
    s = s.replace('<AUTHOR>', 'Greedy AI')
    return s

接下来加载函数并且直接转为我们需要的格式

In [6]:
import json
import random
def load_self_cong_data(filename):
    data_list = []
    for d in json.load(open(filename, "r", encoding="utf-8")):
        d["instruction"] = replace_name(d["instruction"])
        d["output"] = replace_name(d["output"])
        data_list.append({
            "type": "chatml",
            "messages": [
                {
                    "role": "system",
                    "content": "You are a helpful assistant."
                },
                {
                    "role": "user",
                    "content": d["instruction"]
                },
                {
                    "role": "assistant",
                    "content": d["output"]
                }
            ],
            "source": "self-made"
        })
    return data_list

In [7]:
self_cong_data = load_self_cong_data("../../data/llm/self_cogniton.json")
print(self_cong_data[1])

{'type': 'chatml', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '您好，我是 智能医学客服机器人，一个由 Greedy AI 打造的人工智能助手，请问有什么可以帮助您的吗？'}], 'source': 'self-made'}


接下来加载前面的医疗对话数据，并shuffle打乱顺序。

In [8]:
import random
format_data_list = prepare_message(data_list)
random.shuffle(format_data_list)
format_data_list = self_cong_data + format_data_list   # 合并数据集
print(format_data_list[0])
print(format_data_list[100])

{'type': 'chatml', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '您好，我是 智能医学客服机器人，一个由 Greedy AI 开发的 AI 助手，很高兴认识您。请问我能为您做些什么？'}], 'source': 'self-made'}
{'type': 'chatml', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '大夫宝宝四个月了发烧四天吃退烧药老是反复今天总是哭闹怎么回事怎么办呢麻烦您解答下 以前也发烧过吃退烧药几天好了，怎么办好呢'}, {'role': 'assistant', 'content': '发烧是许多疾病都会有的症状，单纯把烧退下来不等于病被治好了，一定要把引起发烧的病治好了，体温方才能真正地退下来。体温在三十八点五度以上时，可用布洛芬混悬滴剂或对乙酰氨基酚缓释片退热治疗，体温在三十八点五度以下时，勤些用温水擦浴以降温，或用退热贴敷额能起到退热护脑的效用。'}], 'source': 'medical'}


下面划分训练集和验证集，直接将前84000条划分为训练集，剩下的作为验证集，

In [9]:
train_data = format_data_list[:84000]
test_data = format_data_list[84000:]
print("train data size:", len(train_data))
print("test data size:", len(test_data))

train data size: 84000
test data size: 424


#### 配置模型
在导入模型之前，我们需要设置一些Lora和量化的配置参数。
<br>
因为我们在这个项⽬中要使用 QLoRA，而且要 quantize 成 4bit，所以这些信息要提前做好配置。这样模型导入的时候，就会被按照我们的要求转换为 4bit 的量化模型。

In [10]:
compute_dtype = getattr(torch, "bfloat16")   # 使用 bf16 格式进行计算，节省显存、加速计算
quant_config = BitsAndBytesConfig(    # 配置模型量化参数，采用 nf4 类型的 4bit 量化，采用双量化技术减小模型大小。具体参考 qlora。
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)

#### 加载模型

In [11]:
model_path = "../qwen/Qwen2-7B-Instruct"
original_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=compute_dtype,
    device_map=device_map,
    quantization_config=quant_config,   # 填入量化参数，可以大幅节省显存
    trust_remote_code=True
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:10<00:00,  2.75s/it]


#### 加载tokenizer
用于将字符串形式的文本转换为 token 的 id。所以我们需要导入适合当前模型的 tokenizer，并进行后续 tokenize 的操作。

In [12]:
# padding_size是指当一个batch内某些序列长度不足时，在哪一侧填充。将batch内所有序列填充为相同长度，可以达到更好的训练效率。
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True, padding_side="right",)
# 将分词器的填充令牌（pad_token）设置为与结束符令牌（eos_token）相同。在对话场景下，这样的做法有助于模型更容易学习到什么时候应该结束
tokenizer.pad_token_id = tokenizer.eos_token_id  

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


#### 测试导入的模型效果

In [13]:
from transformers import set_seed
seed = 42
set_seed(seed)
index = 10

prompt = "你好"
prompt1 = "孩子积食了怎么办？"
prompt2 = "孩子身上长疹子了是啥原因呢"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt2}
]
model_inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt').to(device)
generated_ids = original_model.generate(model_inputs, max_new_tokens=512)

generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs, generated_ids)]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

孩子身上长疹子的原因可能有很多，以下是一些常见的可能性：

1. **过敏反应**：可能是对食物、药物、衣物材质、宠物、花粉等过敏导致的皮疹。
2. **病毒感染**：如水痘、手足口病等病毒性感染可以引起皮肤疹子。
3. **细菌感染**：如脓疱疮、疖子等细菌感染也可能导致皮肤出现红疹。
4. **真菌感染**：如湿疹、股癣、头癣等由真菌引起的皮肤病。
5. **接触性皮炎**：接触某些刺激性物质或过敏原后，皮肤接触部位出现红疹。
6. **荨麻疹**：一种突发性的皮肤反应，通常由过敏或其他刺激引起，表现为红色或肿胀的皮疹。
7. **湿疹**：一种常见的慢性皮肤炎症，表现为皮肤干燥、红斑、瘙痒和脱屑。
8. **药物反应**：某些药物可能导致皮肤疹子作为副作用。

如果孩子身上长疹子，建议及时就医，由专业医生进行诊断和治疗。医生可能会根据症状、体征以及必要的检查（如血液检查、皮肤刮片检查）来确定疹子的具体原因，并给出相应的治疗方案。自行处理时应避免使用未经医生指导的药物，以免加重病情。


可以看到Qwen2模型本身就有医学知识问答能力了，我们可以观察下经过微调它的回答会不会有变化，是效果更好还是更差了。

#### 数据预处理
预处理主要分为两个部分：
1. 对于给定的数据，进行格式化，使得满足 instruction 数据所需要的格式。
2. 把文本转换成 token ids 的形式。

In [14]:
from transformers.trainer_pt_utils import LabelSmoother
TEMPLATE = "{% for message in messages %}{% if loop.first and messages[0]['role'] != 'system' %}{{ '<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n' }}{% endif %}{{'<|im_start|>' + message['role'] + '\n' + message['content']}}{% if loop.last %}{{ '<|im_end|>'}}{% else %}{{ '<|im_end|>\n' }}{% endif %}{% endfor %}"
IGNORE_TOKEN_ID = LabelSmoother.ignore_index
def preprocess(
        messages,
        tokenizer: AutoTokenizer,
        max_len: int
    ):
    texts = []
    for i, msg in enumerate(messages):
        texts.append(
            tokenizer.apply_chat_template(
                msg,
                chat_template=TEMPLATE,
                tokenize=True,
                add_generation_prompt=False,
                padding="max_length",
                max_length=max_len,
                truncation=True,
            )
        )
    input_ids = torch.tensor(texts, dtype=torch.int)
    target_ids = input_ids.clone()
    target_ids[target_ids == tokenizer.pad_token_id] = IGNORE_TOKEN_ID
    attention_mask = input_ids.ne(tokenizer.pad_token_id)
    return dict(
        input_ids=input_ids,
        labels=target_ids,
        attention_mask=attention_mask
    )

#### 构造数据集
定义模型finetune需要的数据集对象

In [15]:
from torch.utils.data import Dataset
class SupervisedDataset(Dataset):
    """Dataset for supervised fine-tuning."""
    def __init__(self, raw_data, tokenizer, max_len: int):
        super(SupervisedDataset, self).__init__()
        print("Formatting inputs...")
        sources = [example["messages"] for example in raw_data]
        data_dict = preprocess(sources, tokenizer, max_len)
        self.input_ids = data_dict["input_ids"]
        self.labels = data_dict["labels"]
        self.attention_mask = data_dict["attention_mask"]
        print("Formatting done...")
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, i):
        return dict(
            input_ids=self.input_ids[i],
            labels=self.labels[i],
            attention_mask=self.attention_mask[i],
        )

用SupervisedDataset包装训练数据和验证数据集：

In [16]:
train_dataset = SupervisedDataset(train_data[:2000], tokenizer, max_len=1024)   # 取前1000条数据作为示例
test_dataset = SupervisedDataset(test_data, tokenizer, max_len=1024)
print(train_data[0])

Formatting inputs...
Formatting done...
Formatting inputs...
Formatting done...
{'type': 'chatml', 'messages': [{'role': 'system', 'content': 'You are a helpful assistant.'}, {'role': 'user', 'content': '你好'}, {'role': 'assistant', 'content': '您好，我是 智能医学客服机器人，一个由 Greedy AI 开发的 AI 助手，很高兴认识您。请问我能为您做些什么？'}], 'source': 'self-made'}


#### 构造用于 finetune 的模型（模型实例化）

In [17]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
config = LoraConfig(  # lora 参数配置
    r=32, #Rank  64 32 
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "up_proj", "gate_proj", "down_proj"],  # 指定了要在模型中插入 LoRA 适配器的模块，也就是需要训练的参数
    bias="none",
    lora_dropout=0.05, # Conventional
    task_type="CAUSAL_LM",
)
# 1 - use q_lora, Using the prepare_model_for_kbit_training method from PEFT
original_model = prepare_model_for_kbit_training(original_model, use_gradient_checkpointing=True)
peft_model = get_peft_model(original_model, config)
# Print peft trainable params
peft_model.print_trainable_parameters()  # 统计参数
# use gradient_checkpointing
peft_model.enable_input_require_grads()

trainable params: 80,740,352 || all params: 7,696,356,864 || trainable%: 1.0491


#### 微调训练参数配置

In [18]:
output_dir = './checkpoints_self_cong-qwen2-7b/'   # 模型训练过程中生成结果的保存路径
peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=10,
    per_device_train_batch_size=1,                 # 每个设备上的 batch size
    gradient_accumulation_steps=1,                 # 梯度累积
    num_train_epochs=1,                            # 训练轮数
    learning_rate=2e-4,                            # 学习率
    optim="paged_adamw_8bit",                      # 指定优化器
    logging_steps=100,                             # 日志保存间隔
    logging_dir="./logs", 
    save_strategy="steps",                         # 模型保存策略，按照 steps
    max_steps=1000,                                # 最大训练步数
    save_steps=100,                                # 模型保存间隔
    evaluation_strategy="steps",                   # 模型评估间隔策略，按照 steps
    eval_steps=1001,                               # 模型评估间隔，模型评估会占用大量显存，容易OOM，设置为大于 max_steps，则训练过程中不评估
    do_eval=True,                                  # 是否训练时做评估，试了一次不起作用就没再试
    gradient_checkpointing=True,                   # 启用梯度检查点节省显存
    report_to="none",
    overwrite_output_dir = 'True',                 # 如果目录存在是否覆盖
    group_by_length=True,                          # 将类似长度的样本分到一起，减少填充、提高训练效率
)
peft_model.config.use_cache = False
peft_trainer = Trainer(
    model=peft_model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=peft_training_args,
    #data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

Detected kernel version 4.15.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


#### 开启训练
训练结束后可以去指定的 output_dir 中查看训练生成的结果。

In [19]:
torch.cuda.empty_cache()
peft_trainer.train()

/opt/envs/miniconda3/envs/pytorch2.1/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss


/opt/envs/miniconda3/envs/pytorch2.1/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ../qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/envs/miniconda3/envs/pytorch2.1/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/envs/miniconda3/envs/pytorch2.1/lib/python3.10/site-packages/peft/utils/save_and_load.py:195: UserWarning: Could not find a config file in ../qwen/Qwen2-7B-Instruct - will assume that the vocabulary was not modified.
  warnings.warn(
/opt/envs/miniconda3/envs/pytorch2.1/lib/

TrainOutput(global_step=1000, training_loss=1.632033950805664, metrics={'train_runtime': 1701.5623, 'train_samples_per_second': 1.175, 'train_steps_per_second': 0.588, 'total_flos': 8.7875905388544e+16, 'train_loss': 1.632033950805664, 'epoch': 1.0})


统计参数，非必须

In [ ]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\n \
            all model parameters: {all_model_params}\n \
            percentage of trainable model parameters: \
            {100 * trainable_model_params / all_model_params:.2f}%"

print_number_of_trainable_model_parameters(original_model)

'trainable model parameters: 80740352\n             all model parameters: 4433712640\n             percentage of trainable model parameters:             1.82%'

#### 模型推理
接下来，我们如果想加载训练好的checkpoint，需要进行参数的合并，也就是参数(base 模型) + 参数（lora）。
<br>
可以重启一下内核，释放显存。否则显存小的话容易OOM。重启的话再执行一下最开始的导入包的cell。

先加载基础模型

In [2]:
compute_dtype = getattr(torch, "bfloat16")
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=True,
)
model_path = "../qwen/Qwen2-7B-Instruct"
original_model = AutoModelForCausalLM.from_pretrained(
    model_path,
    torch_dtype=compute_dtype,
    device_map=device_map,
    quantization_config=quant_config,
    trust_remote_code=True
)
tokenizer = AutoTokenizer.from_pretrained(model_path, use_fast=False, trust_remote_code=True)
tokenizer.pad_token_id = tokenizer.eos_token_id

Loading checkpoint shards: 100%|██████████| 4/4 [00:09<00:00,  2.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


加载微调后保存的 lora 模型

In [3]:
from peft import PeftModel
output_dir = './checkpoints_self_cong-qwen2-7b/'
ft_model = PeftModel.from_pretrained(
    original_model,
    output_dir + '/checkpoint-1000',
    torch_dtype=compute_dtype,
    device_map=device_map,
    quantization_config=quant_config
)

接下来我们测试下微调后的模型：

In [4]:
# prompt = "你好"
prompt = "孩子身上长疹子了是啥原因呢"
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
]
model_inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt').to('cuda')
generated_ids = ft_model.generate(model_inputs, max_new_tokens=512)

generated_ids = [output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs, generated_ids)]
response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(response)

你好，根据你的描述，一般考虑是过敏引起的，可以口服开瑞坦和维生素C片，外用炉甘石洗剂清洗患处。注意卫生，多喝水，不吃辛辣食物。祝你健康！，小儿保健问题一直困扰着家长们，在治疗此病时，考虑采取一般治疗，父母只有利用正确的方法才能尽快帮助儿童治疗好疾病，才不会对他们的成长发育造成影响，所以希望家长们一定要关注孩子的情况，及时带孩子就医治疗。<|endoftext|>


可以看到机器的自我认知已经被更改，并且对于问题的回答风格和之前的已经发生了变化。如果机器资源充足，可以加载更多的数据训练更长时间观察效果。